# This is just a notebook to visualise 1kHz filtered raw data

## Setup everything

### Import packages

In [1]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
import fnmatch
import os
import mmap
import xarray as xr
import dask.array as da
%matplotlib widget
from IPython.display import display
from ipyfilechooser import FileChooser
import ipywidgets as widgets

from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
import ephyviewer

### Define general functions

In [2]:
# Function to find files containing a specific string
def find_files_with_string(folder_path, search_string):
    matching_file = []
    # Traverse the folder to find files
    for root, _, files in os.walk(folder_path):
        for file in files:
            if fnmatch.fnmatch(file, f"*{search_string}*"):
                matching_file=os.path.join(root, file)
    return matching_file

### Choose experiment

In [3]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"
try:
    %store -r dpath
except:
    print("data not in store")
    dpath ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"


# Set up Initial Basic Parameters#
minian_path = "."

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>OpenEphys Folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/recording8', filename='', title='<b>OpenEphys …

In [4]:
animals = ['BOB', 'Blue', 'Pink']
AnimalID = ''
try:
    %store -r AnimalID
except:
    print("Animal not in store")
    AnimalID =animals[0]


def updateAnimal(widget):
    AnimalID = widget.new
    %store AnimalID

wAnimal = widgets.Dropdown(
    options=animals,
    value=AnimalID,
    description='Animals:',
    disabled=False,
)
wAnimal.observe(updateAnimal, 'value')

display(wAnimal)


Dropdown(description='Animals:', options=('BOB', 'Blue', 'Pink'), value='BOB')

## Load LFPs data

In [5]:
folderpath = Path(dpath)
numchannels=32

if find_files_with_string(folderpath,  ".bin"): #Bonsai
    matching_file = find_files_with_string(folderpath, ".bin")
    All = np.fromfile(matching_file, dtype=np.uint16)
    All = All - int(65535/2)
    All = All.astype(np.int16)
    if 'sommeil' in dpath:
        numchannels=64
    All = All.reshape(-1,numchannels)
    print('File loaded: OE32channels.bin,', numchannels, 'channels')

elif find_files_with_string(folderpath,  "continuous.dat"): #OpenEphys
    matching_file = find_files_with_string(folderpath, "continuous.dat")
    All = np.fromfile(matching_file, dtype=np.int16)

    #All =np.memmap(matching_file, dtype='int16', mode='c')
    
    #All = All - int(65535/2)
    #All = All.astype(np.int16)    
    if 'sommeil' in dpath:
        numchannels=64
    All = All.reshape(-1,numchannels)
    print('File loaded: continuous.dat,', numchannels, 'channels')

elif find_files_with_string(folderpath,  "RawDataChannelExtractedDS.npy"): #OpenEphys Gaelle's Data
    matching_file = find_files_with_string(folderpath, "RawDataChannelExtractedDS.npy")
    All = np.load(matching_file, mmap_mode= 'r')
    
    #All = All.reshape(-1,numchannels)
    print('File loaded: RawDataChannelExtractedDS.npy,', numchannels, 'channels')

All.shape

File loaded: continuous.dat, 32 channels


(47027712, 32)

In [6]:
All = np.fromfile(matching_file, dtype=np.int16)
numchannels=64
All = All.reshape(-1,numchannels)

In [7]:
# Filtre parameter:
f_lowcut = 1
f_hicut = 50.
fs = 1000
nyq = 0.5 * fs
N = 4                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')


In [8]:
match AnimalID:
    case 'BOB':
        #BOB
        S1=All[:, 5]-All[:,6]
        PFC=All[:, 2]-All[:,3]
        CA1=All[:, 29]-All[:,30]
        EMG=All[:,27]
        combined = np.stack([S1, PFC, CA1, EMG/10], axis = 1)
    case 'Blue':
        #Blue
        PFC=All[:, 14]-All[:,15]
        PFC = signal.filtfilt(b, a, PFC)
        S1=All[:, 8]-All[:,9]
        S1 = signal.filtfilt(b, a, S1)
        EMG=All[:,20]
        combined = np.stack([S1,PFC, EMG/10], axis = 1)
    case 'Pink':
        #Pink
        PFC=All[:, 26+32]-All[:,27+32]
        EMG=All[:,9+32]
        S1=All[:, 0+32]-All[:,1+32]
        combined = np.stack([S1,PFC, EMG/10], axis = 1)
    case _:
        print("no known animal defined")
        combined = All

### Visualise All

In [9]:
%gui qt 
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers

win = MainViewer()
source = InMemoryAnalogSignalSource(combined, sample_rate, t_start)
view1 = TraceViewer(source=source, name='trace')

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

#create a time freq viewer conencted to the same source
view2 = TimeFreqViewer(source=source, name='tfr')

view2.params['show_axis'] = True
view2.params['timefreq', 'deltafreq'] = 1
view2.params['timefreq', 'f_stop'] = 200
#view2.by_channel_params['ch0', 'timefreq', 'f_stop'] = 20
#view2.by_channel_params['ch1', 'timefreq', 'f_stop'] = 150
view2.by_channel_params['ch0', 'visible'] = False
view2.by_channel_params['ch1', 'visible'] = True
view2.by_channel_params['ch1', 'clim'] = 100

#add them to mainwindow
win.add_view(view1)
win.add_view(view2)


#Run
win.show()
app.exec_()

/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_7951/3710607648.py:35: DeprecationWarning: 'exec_' will be removed in the future. Use 'exec' instead.
  app.exec_()
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=460.414,203.424 gbl=460.414,203.424 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-460.414,-203.424 last=-460.414,-203.424 Δ 460.414,203.424) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=670.85,719.365 gbl=670.85,719.365 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-670.85,-719.365 last=-670.85,-719.365 Δ 670.85,719.365) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=403.318,417.314 gbl=403.318,417.314 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-403.318,-417.314 last=-403.318,-417.314 Δ 403.318,417.314) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=261.754,203.88 gbl=261.754,203.88 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-261.754,-203.88 last=-261.754,

0